In [32]:
# This scrapping code comes from Dharun Suryaa
# I modified it to get Jet2 reviews data

from bs4 import BeautifulSoup
import requests
import re

In [5]:
url ='https://www.airlinequality.com/airline-reviews/jet2-com'
page= requests.get(url)
soup= BeautifulSoup(page.content,"html.parser")

In [6]:
rating, header,author, date, place, verification, content, collect=[],[],[],[],[],[],[],[]
urls=[url]
c=1
while(soup!=None):
    articles = soup.find_all("article", attrs={'class':re.compile('comp comp_media-review-rated list-item media position-content review-[0-9]*')})
    for article in articles:
        if(article.find("div",class_='rating-10').find('span') is None):
            rating.append(None)
        else:
            rating.append(article.find("div",class_='rating-10').find('span').text)
        if(article.find("div",class_='body').find('h2') is None):
            header.append(None)
        else:
            header.append(article.find("div",class_='body').find('h2').text.strip("\""))
        if(article.find("div",class_='body').find('span',itemprop='author').find('span') is None):
            author.append(None)
        else:
            author.append(article.find("div",class_='body').find('span',itemprop='author').find('span').text)
        if(article.find("div",class_='body').find('time',itemprop='datePublished') is None):
            date.append(None)
        else:
            date.append(article.find("div",class_='body').find('time',itemprop='datePublished')['datetime'])
        if(re.search(r".*(\(.*\)).*",article.find("div",class_='body').find('h3').text.strip()) is None):
            place.append(None)
        else: 
            place.append(re.search(r".*(\(.*\)).*",article.find("div",class_='body').find('h3').text.strip()).group(1).strip('()'))
        content.append(article.find("div",class_='text_content').text.strip())
        tr= article.find("div",class_='body').find('table', class_='review-ratings').findAll(lambda tag: tag.name=='tr')
        table_dict={}
        for row in tr:
            key= row.find('td',attrs={'class':re.compile('review-rating-header.*')}).text
            if(row.find('td',class_='review-value')==None):
                if(not [i.text for i in row.find('td',attrs={'class':re.compile('review-rating-stars.*')}).findAll('span',class_='star fill')]):
                    value=None
                else:
                    value=max([i.text for i in row.find('td',attrs={'class':re.compile('review-rating-stars.*')}).findAll('span',class_='star fill')])
            else:
                value= row.find('td',class_='review-value').text
            table_dict[key]=value
        collect.append(table_dict)
    next_article=soup.find('article', class_='comp comp_reviews-pagination querylist-pagination position-')
    c+=1
    next_url="https://www.airlinequality.com"+"/airline-reviews/jet2-com/page/"+str(c)+"/"
    urls.append(next_url)
    page= requests.get(next_url)
    soup= BeautifulSoup(page.content,"html.parser")
    if(c==367):
        break

In [7]:
import pandas as pd
import numpy as np

In [8]:
airline_review = pd.DataFrame({"rating":rating, 
                               "header":header, 
                               "author":author, 
                               "date":date,
                               "place":place,
                               "content":content, 
                               "collect":collect})

In [9]:
airline_review.dropna(inplace=True)

In [10]:
airline_review.rating=airline_review.rating.astype(int)

In [11]:
airline_review.date=pd.to_datetime(airline_review.date)

In [12]:
new_airline_review=airline_review.merge(pd.json_normalize(airline_review.collect).rename(columns={
    'Aircraft':'aircraft',
    'Type Of Traveller':'traveller_type',
    'Seat Type':'seat_type',
    'Route':'route',
    'Date Flown':'date_flown',
    'Seat Comfort':'seat_comfort',
    'Cabin Staff Service':'cabin_staff_service', 
    'Food & Beverages':'food_beverages',
    'Ground Service':'ground_service',
    'Value For Money':'value_for_money',
    'Recommended':'recommended',
    'Wifi & Connectivity':'wifi_connectivity',
    'Inflight Entertainment':'entertainment'    
}), left_index=True, right_index=True).drop(columns=['collect'])

In [13]:
new_airline_review.content = new_airline_review.content.str.split('|').apply(lambda x: x[1] if len(x)>=2 else x[0])

In [14]:
new_airline_review.head()

,rating,header,author,date,place,content,aircraft,traveller_type,seat_type,route,date_flown,seat_comfort,cabin_staff_service,food_beverages,ground_service,value_for_money,recommended,entertainment,wifi_connectivity
0,5,I got the unfriendly fare,L Kirk,2024-01-16,United Kingdom,Swift check in service with plenty of staff ...,Boeing 737,Solo Leisure,Economy Class,Manchester to Barcelona,July 2023,1,4,5,5,1,yes,NaN,NaN
1,2,ground and cabin staff are abrupt,Lorraine Atkinson,2024-01-02,United Kingdom,Jet2 ground and cabin staff are abrupt and un...,NaN,Couple Leisure,Economy Class,Manchester to Lanzarote,December 2023,2,1,NaN,1,3,no,NaN,NaN
2,10,"Crew friendly & efficient""",Karl Whitehead,2023-12-02,United Kingdom,First time flying with Jet2 they didn't dis...,A321 neo,Family Leisure,Economy Class,Manchester to Fuerterventura,November 2023,4,5,5,5,5,yes,NaN,NaN
3,5,The seats are really uncomfortable,R Savage,2023-10-29,United Kingdom,So we flew out at 6.30 am. The seats are rea...,NaN,Couple Leisure,Economy Class,Athens to Manchester,October 2023,1,2,1,3,1,no,1,1
4,8,a long rung above the competition,2 reviews,2023-10-17,United Kingdom,In the UK it is pretty difficult to get a f...,Boeing 737-800,Couple Leisure,Economy Class,Funchal to East Midlands,October 2023,1,4,NaN,4,5,yes,NaN,NaN


In [15]:
new_airline_review.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 476 entries, 0 to 525
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   rating               476 non-null    int64         
 1   header               476 non-null    object        
 2   author               476 non-null    object        
 3   date                 476 non-null    datetime64[ns]
 4   place                476 non-null    object        
 5   content              476 non-null    object        
 6   aircraft             97 non-null     object        
 7   traveller_type       237 non-null    object        
 8   seat_type            475 non-null    object        
 9   route                235 non-null    object        
 10  date_flown           235 non-null    object        
 11  seat_comfort         463 non-null    object        
 12  cabin_staff_service  463 non-null    object        
 13  food_beverages       363 non-null  

In [16]:
new_airline_review.rating.mean()

6.7478991596638656

In [17]:
new_airline_review['trip_verified']=airline_review.content.str.split('|').apply(lambda x: x[0] if len(x)>=2 else None)

In [18]:
new_airline_review.drop(columns={'wifi_connectivity'}, inplace=True)

In [19]:
new_airline_review.date_flown = pd.to_datetime(new_airline_review.date_flown)

In [20]:
new_airline_review.seat_comfort=new_airline_review.seat_comfort.fillna(-1)
new_airline_review.seat_comfort=new_airline_review.seat_comfort.astype(int)
new_airline_review.cabin_staff_service=new_airline_review.cabin_staff_service.fillna(-1)
new_airline_review.cabin_staff_service=new_airline_review.cabin_staff_service.astype(int)
new_airline_review.food_beverages=new_airline_review.food_beverages.fillna(-1)
new_airline_review.food_beverages=new_airline_review.food_beverages.astype(int)
new_airline_review.ground_service=new_airline_review.ground_service.fillna(-1)
new_airline_review.ground_service=new_airline_review.ground_service.astype(int)
new_airline_review.value_for_money=new_airline_review.value_for_money.fillna(-1)
new_airline_review.value_for_money=new_airline_review.value_for_money.astype(int)
new_airline_review.entertainment=new_airline_review.entertainment.fillna(-1)
new_airline_review.entertainment=new_airline_review.entertainment.astype(int)

In [21]:
new_airline_review.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 476 entries, 0 to 525
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   rating               476 non-null    int64         
 1   header               476 non-null    object        
 2   author               476 non-null    object        
 3   date                 476 non-null    datetime64[ns]
 4   place                476 non-null    object        
 5   content              476 non-null    object        
 6   aircraft             97 non-null     object        
 7   traveller_type       237 non-null    object        
 8   seat_type            475 non-null    object        
 9   route                235 non-null    object        
 10  date_flown           235 non-null    datetime64[ns]
 11  seat_comfort         476 non-null    int64         
 12  cabin_staff_service  476 non-null    int64         
 13  food_beverages       476 non-null  

In [22]:
new_airline_review.head()

,rating,header,author,date,place,content,aircraft,traveller_type,seat_type,route,date_flown,seat_comfort,cabin_staff_service,food_beverages,ground_service,value_for_money,recommended,entertainment,trip_verified
0,5,I got the unfriendly fare,L Kirk,2024-01-16,United Kingdom,Swift check in service with plenty of staff ...,Boeing 737,Solo Leisure,Economy Class,Manchester to Barcelona,2023-07-01,1,4,5,5,1,yes,-1,Not Verified
1,2,ground and cabin staff are abrupt,Lorraine Atkinson,2024-01-02,United Kingdom,Jet2 ground and cabin staff are abrupt and un...,NaN,Couple Leisure,Economy Class,Manchester to Lanzarote,2023-12-01,2,1,-1,1,3,no,-1,Not Verified
2,10,"Crew friendly & efficient""",Karl Whitehead,2023-12-02,United Kingdom,First time flying with Jet2 they didn't dis...,A321 neo,Family Leisure,Economy Class,Manchester to Fuerterventura,2023-11-01,4,5,5,5,5,yes,-1,Not Verified
3,5,The seats are really uncomfortable,R Savage,2023-10-29,United Kingdom,So we flew out at 6.30 am. The seats are rea...,NaN,Couple Leisure,Economy Class,Athens to Manchester,2023-10-01,1,2,1,3,1,no,1,Not Verified
4,8,a long rung above the competition,2 reviews,2023-10-17,United Kingdom,In the UK it is pretty difficult to get a f...,Boeing 737-800,Couple Leisure,Economy Class,Funchal to East Midlands,2023-10-01,1,4,-1,4,5,yes,-1,✅ Trip Verified


In [24]:
new_airline_review.rating.mean()

6.7478991596638656

In [25]:
new_airline_review.trip_verified.unique()

array(['Not Verified ', '✅ Trip Verified ', None, '✅ Verified Review ',
       '❎ Unverified '], dtype=object)

In [26]:
new_airline_review.trip_verified.value_counts()

✅ Trip Verified       63
Not Verified          45
✅ Verified Review     34
❎ Unverified           1
Name: trip_verified, dtype: int64

In [27]:
def clean_trip(x):
    if(x=='Not Verified ' or x=='❎ Not Verified ' or x=='❎ Unverified '):
        return 'Not Verified'
    elif(x=='✅ Trip Verified ' or x=='✅ Verified Review '):
        return 'Verified'
    else:
        return None

new_airline_review.trip_verified=new_airline_review.trip_verified.apply(clean_trip)

In [28]:
new_airline_review.columns

Index(['rating', 'header', 'author', 'date', 'place', 'content', 'aircraft',
       'traveller_type', 'seat_type', 'route', 'date_flown', 'seat_comfort',
       'cabin_staff_service', 'food_beverages', 'ground_service',
       'value_for_money', 'recommended', 'entertainment', 'trip_verified'],
      dtype='object')

In [29]:
new_airline_review.drop_duplicates(inplace=True)

In [30]:
new_airline_review.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 476 entries, 0 to 525
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   rating               476 non-null    int64         
 1   header               476 non-null    object        
 2   author               476 non-null    object        
 3   date                 476 non-null    datetime64[ns]
 4   place                476 non-null    object        
 5   content              476 non-null    object        
 6   aircraft             97 non-null     object        
 7   traveller_type       237 non-null    object        
 8   seat_type            475 non-null    object        
 9   route                235 non-null    object        
 10  date_flown           235 non-null    datetime64[ns]
 11  seat_comfort         476 non-null    int64         
 12  cabin_staff_service  476 non-null    int64         
 13  food_beverages       476 non-null  

In [31]:
new_airline_review.to_csv("Jet2_reviews.csv")